In [1]:
# As third_reg, but not trying to predict around the peak.

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
def gen_one_data():
    """Generates a random feature/label combination.
    
    Here the features are the solution on a coarse grid and the log of the
    solution values, and the label is the log of the solution on a fine grid.
    """
    
    # Random solution to the CH equation
    c = np.random.uniform(0, 10)
    peakon = Peakon(c=c)
    # Random location
    t = np.random.uniform(0, 40)
    x = c * t + tools.random_function(lambda: np.random.uniform(-5, -1), lambda:np.random.uniform(1, 5))
    
    # Grids at the location
    cg = coarse_grid((t, x))
    fg = fine_grid((t, x))
    # Features: the solution on the coarse grid
    X = peakon.on_grid(cg)
    # Labels: the solution on the fine grid
    y = peakon.on_grid(fg)
    
    return np.concatenate([X, np.log(X)]), np.log(y)

# Note that bilinear interpolation is now interpolating log-values to find 
# log-values.
class BilinearInterp(BilinearInterpBase):
    @classmethod
    def _predict(cls, Xi):
        returnval = []
        # Translation doesn't matter at this point so WLOG the fine grid is
        # around 0, 0. (cls._interp makes the same assumption; these assumptions
        # must be consistent)
        for point in fine_grid((0, 0)):
            returnval.append(cls._interp(Xi, point))
        return returnval

In [5]:
# DNN hyperparameters
hidden_units = [1000] * 20
logits = 121  # = (_fine_grid_fineness.t + 1) * (_fine_grid_fineness.x + 1)
              # i.e. the number of fine grid points.
drop_rate = 0.4

# Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir = './saved_models/fourth_reg/'
for layer in hidden_units:
    model_dir += '{}_'.format(layer)
model_dir += '{}_'.format(logits)
model_dir += 'D0{}_'.format(int(drop_rate * 10))
# model_dir += 'BS{}_'.format(batch_size)
# model_dir += 'BR{}'.format(batch_reuse)

k_init = tfi.truncated_normal(mean=0, stddev=0.05)
model = Sequential()
for units in hidden_units:
    model.add(tfla.Dense(units=units,
                         activation=ft.partial(tf.nn.leaky_relu, alpha=0.1),
                         kernel_initializer=k_init))
    model.add_train(tfla.Dropout(rate=drop_rate))
model.add(tfla.Dense(units=logits))
model.set_kwargs(model_dir=model_dir,
                 config=tfe.RunConfig(log_step_count_steps=log_steps))

In [6]:
dnn = model.compile()

INFO:tensorflow:Using config: {'_model_dir': './saved_models/fourth_reg/1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_121_D04_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc569fc9cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# Training hyperparameters

batch_size = 50
batch_reuse = 1  # See BatchData for an explanation of batch reuse.
steps = 20000

if TRAIN:
    train_input_fn = BatchData(gen_one_data, batch_size, batch_reuse)
    dnn.train(input_fn=train_input_fn, max_steps=steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/fourth_reg/1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_121_D04_/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into ./saved_models/fourth_reg/1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_121_D04_/model.ckpt.
INFO:tensorflow:loss = 186.2314, step = 2
INFO:tensorflow:global_step/sec: 3.35732
INFO:tensorflow:loss = 0.6234471, step = 1002 (297.859 sec)
INFO:tensorflow:global_step/sec: 3.4404
INFO:tensorflow:loss = 0.45757335, step = 2002 (290.664 sec)
INFO:tensorflow:Saving checkpoints for 2040 into ./saved_models/fourth_reg/1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1000_1

In [ ]:
# Training hyperparameters

batch_size = 500
batch_reuse = 1  # See BatchData for an explanation of batch reuse.
steps = 30000

if TRAIN:
    train_input_fn = BatchData(gen_one_data, batch_size, batch_reuse)
    dnn.train(input_fn=train_input_fn, max_steps=steps)

In [ ]:
testing_data = gen_test_data(gen_one_data)

In [ ]:
test_regressor(testing_data, dnn)

In [ ]:
test_regressor(testing_data, BilinearInterp)